In [2]:
%load_ext autotime
import pandas as pd
import requests
from datetime import datetime
from tqdm import tqdm
import os
import logging

time: 5.27 s (started: 2023-01-22 16:10:04 -05:00)


### Function to get Data from Trade API

Update the parameters as needed based on pdf in API Reference Docs Folder

In [3]:
## get data from 2018 to 2022-11 by HS Code, Country and District


def getCountryDistrictValsHS(code,year,month,dirPath,comm_lvl,summ_lvl2):
    # os.chdir(".apiResponses")
    logTime = str(datetime.now().strftime(r"%Y_%m_%d-%I_%M_%S_%p"))
    logging.basicConfig(filename=os.path.join(dirPath,f'{comm_lvl}_{summ_lvl2}_{code}_{logTime}_download.log'), encoding='utf-8', level=logging.DEBUG)
    if code == 'hs':
        base = f"https://api.census.gov/data/timeseries/intltrade/imports/{code}"
        params = {"get":r','.join(["I_COMMODITY","I_COMMODITY_SDESC","I_COMMODITY_LDESC",
                                "CTY_CODE","DISTRICT",'DIST_NAME',"RP","DUT_VAL_MO",
                                "CON_VAL_MO","CON_QY1_MO","CAL_DUT_MO",
                                "GEN_CHA_MO","GEN_VAL_MO","GEN_CIF_MO","GEN_QY1_MO",'UNIT_QY1','CC_MO',
                                "CNT_CHA_MO","CNT_VAL_MO","CNT_WGT_MO",
                                "AIR_CHA_MO","AIR_VAL_MO","AIR_WGT_MO",
                                "VES_CHA_MO","VES_VAL_MO","VES_WGT_MO"]),
                # "CTY_NAME":['INDIA','CHINA','INDONESIA','VIETNAM','MEXICO','CANADA','BRAZIL','TAIWAN'],
                "YEAR":year,
                "MONTH":month,
                "COMM_LVL":comm_lvl,
                "SUMMARY_LVL2":summ_lvl2,
                "key":"f6b9d2bb24bacdbcc9bb836aa36a2c192fb902e3"}
        temp = requests.get(base,params=params)
    # print(temp.url)
    
        df = pd.DataFrame(temp.json())
        new_header = df.iloc[0] #grab the first row for the header
        df = df[1:] #take the data less the header row
        df.columns = new_header #set the header row as the df header

        # df = df.drop('Unnamed: 0',axis=1)
        df['MONTH'] = df['MONTH'].astype(str).apply(lambda x:str(x).zfill(2))
        df['YearMonth'] = df[['YEAR','MONTH']].astype(str).agg('-'.join,axis=1)
        if 'I_COMMODITY' in df.columns:
            df['I_COMMODITY'] = df['I_COMMODITY'].apply(lambda x:str(x).zfill(10))
            df['HS2'] = df['I_COMMODITY'].apply(lambda x:str(x)[:2])
            df['HS4'] = df['I_COMMODITY'].apply(lambda x:str(x)[:4])
            df['HS6'] = df['I_COMMODITY'].apply(lambda x:str(x)[:6])
            df = df.rename({'I_COMMODITY':'HS10'},axis=1)

# I_COMMODITY_LDESCCTY_CODE

    if code == 'naics':
        base = f"https://api.census.gov/data/timeseries/intltrade/imports/{code}"
        params = {"get":r','.join(["NAICS", "NAICS_LDESC", "NAICS_SDESC", 
                                "CTY_CODE",'CTY_NAME',"DISTRICT",'DIST_NAME',"DUT_VAL_MO",
                                "CON_VAL_MO","CAL_DUT_MO",
                               "GEN_CHA_MO","GEN_VAL_MO","GEN_CIF_MO",'CC_MO',
                                "CNT_CHA_MO","CNT_VAL_MO","CNT_WGT_MO",
                                "AIR_CHA_MO","AIR_VAL_MO","AIR_WGT_MO",
                                "VES_CHA_MO","VES_VAL_MO","VES_WGT_MO"]),
                # "CTY_NAME":['INDIA','CHINA','INDONESIA','VIETNAM','MEXICO','CANADA','BRAZIL'],
                "YEAR":year,
                "MONTH":month,
                "COMM_LVL":comm_lvl,
                "SUMMARY_LVL2":summ_lvl2,
                "key":"f6b9d2bb24bacdbcc9bb836aa36a2c192fb902e3"}
        temp = requests.get(base,params=params)
    # print(temp.url)
    
        df = pd.DataFrame(temp.json())
        new_header = df.iloc[0] #grab the first row for the header
        df = df[1:] #take the data less the header row
        df.columns = new_header #set the header row as the df header

        # df = df.drop('Unnamed: 0',axis=1)
        df['MONTH'] = df['MONTH'].astype(str).apply(lambda x:str(x).zfill(2))
        df['YearMonth'] = df[['YEAR','MONTH']].astype(str).agg('-'.join,axis=1)

        if 'NACIS' in df.columns:
            df['NAICS'] = df['NAICS'].apply(lambda x:str(x).zfill(6))
            df['NA2'] = df['NACIS'].apply(lambda x:str(x)[:2])
            df['NA4'] = df['NACIS'].apply(lambda x:str(x)[:4])
            df = df.rename({'NACIS':'NA6'},axis=1)

    
    fileName = f"{dirPath}/{code}/{params['COMM_LVL']}_{params['SUMMARY_LVL2']}_{year}_{month}.gzip"
    
    df.to_csv(fileName, compression='gzip',index=False)
    logging.info(f"{logTime}_{df.shape} rows,columns saved.! {df.memory_usage().sum()/1e6}")

    return df

time: 5.42 ms (started: 2023-01-22 16:10:09 -05:00)


In [4]:
years = [str(year) for year in range(2019,2022)]
months = [str(month).zfill(2) for month in range(1,13)]
yearMonthFilter = [tuple([year,month])for year in years for month in months]

time: 104 ms (started: 2023-01-22 16:10:09 -05:00)


In [5]:
yearMonthFilter

[('2019', '01'),
 ('2019', '02'),
 ('2019', '03'),
 ('2019', '04'),
 ('2019', '05'),
 ('2019', '06'),
 ('2019', '07'),
 ('2019', '08'),
 ('2019', '09'),
 ('2019', '10'),
 ('2019', '11'),
 ('2019', '12'),
 ('2020', '01'),
 ('2020', '02'),
 ('2020', '03'),
 ('2020', '04'),
 ('2020', '05'),
 ('2020', '06'),
 ('2020', '07'),
 ('2020', '08'),
 ('2020', '09'),
 ('2020', '10'),
 ('2020', '11'),
 ('2020', '12'),
 ('2021', '01'),
 ('2021', '02'),
 ('2021', '03'),
 ('2021', '04'),
 ('2021', '05'),
 ('2021', '06'),
 ('2021', '07'),
 ('2021', '08'),
 ('2021', '09'),
 ('2021', '10'),
 ('2021', '11'),
 ('2021', '12')]

time: 11.7 ms (started: 2023-01-22 16:10:10 -05:00)


In [27]:
for year,month in tqdm(yearMonthFilter):
    getCountryDistrictValsHS(code='naics',
                            year=year,
                            month=month,
                            dirPath='.apiResponses',
                            comm_lvl='NA6',
                            summ_lvl2='NACYDT')

100%|██████████| 96/96 [2:51:05<00:00, 106.93s/it]  

time: 2h 51min 5s (started: 2023-01-22 01:11:33 -05:00)


In [7]:
for year,month in tqdm(yearMonthFilter):
    getCountryDistrictValsHS(code='hs',
                            year=year,
                            month=month,
                            dirPath='.apiResponses',
                            comm_lvl='HS10',
                            summ_lvl2='HSCYDT')

  0%|          | 0/36 [01:10<?, ?it/s]


KeyboardInterrupt: 

time: 1min 10s (started: 2023-01-22 16:17:20 -05:00)
